*Загрузим необходимые пакеты*

In [1]:
%pip install -q torch lightning transformers datasets trl wandb transformers

Note: you may need to restart the kernel to use updated packages.


# *Level 1*

*Установим модель и датасет*

In [2]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset

/home/hxrt_mx/ml/notebooks/smollm2-reinforce-alignment/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
checkpoint = "HuggingFaceTB/SmolLM2-135M-Instruct"

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.padding_side = "left"
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
	num_labels=1,
).to(device)

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at HuggingFaceTB/SmolLM2-135M-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
dataset_name = "juyoungml/HelpSteer2-binarized"

ds = load_dataset(dataset_name)

train_ds = ds["train"]
val_ds = ds["validation"]
ds

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected', 'chosen_score', 'rejected_score', 'chosen_rationale', 'rejected_rationale', 'score_diff', 'difficulty'],
        num_rows: 7224
    })
    validation: Dataset({
        features: ['prompt', 'chosen', 'rejected', 'chosen_score', 'rejected_score', 'chosen_rationale', 'rejected_rationale', 'score_diff', 'difficulty'],
        num_rows: 373
    })
})

In [5]:
def format_ds(example):
    user_msg = example['prompt']
    chosen = example['chosen']
    rejected = example['rejected']

    prompt = f"<|im_start|>user\n{user_msg}<|im_end|>\n<|im_start|>assistant\n"

    return {
		"prompt": prompt,
		"chosen": f"{prompt}{chosen}<|im_end|>",
		"rejected": f"{prompt}{rejected}<|im_end|>",
    }

In [6]:
print(tokenizer.special_tokens_map)

{'bos_token': '<|im_start|>', 'eos_token': '<|im_end|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|im_end|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}


In [7]:
train_ds = train_ds.map(format_ds, remove_columns=train_ds.column_names)
val_ds = val_ds.map(format_ds, remove_columns=val_ds.column_names)

*Перейдем к обучению reward модели*

In [8]:
from trl import RewardTrainer, RewardConfig

In [9]:
reward_conf = RewardConfig(
    output_dir="reward_model",
    num_train_epochs=1,
    learning_rate=5e-5,
    fp16=True,
    max_length=512,
    eval_strategy="steps",
    eval_steps=50,
)

trainer = RewardTrainer(
    model=model,
    args=reward_conf,
    processing_class=tokenizer,
    train_dataset=train_ds,
    eval_dataset=val_ds,
)

In [10]:
trainer.train()
trainer.save_model(reward_conf.output_dir)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


wandb: Currently logged in as: 671342ihxrxmx (671342ihxrxmx-iu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
50,0.719600,0.689822,0.513966
100,0.659500,0.692218,0.592179
150,0.655100,0.676410,0.597765
200,0.613000,0.652616,0.620112
250,0.601000,0.642865,0.614525
300,0.630600,0.643826,0.612360
350,0.607100,0.621478,0.608939
400,0.601200,0.619745,0.608939


/home/hxrt_mx/ml/notebooks/smollm2-reinforce-alignment/.venv/lib/python3.11/site-packages/trl/trainer/utils.py:815: UserWarning: There are 1 out of 179 instances where the predictions for both options are equal. These instances are ignored in the accuracy computation.
  warnings.warn(
wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


*Перейдем к реализации reinforce \w baseline и дообучению sft с помощью него*

In [11]:
from transformers import AutoModelForCausalLM
from torch.optim import AdamW
from torch.utils.data import DataLoader
from lightning.pytorch.loggers import WandbLogger
from lightning import LightningModule, LightningDataModule, Trainer

*Воспользуемся lightning*

In [12]:
class ReinforceDataModule(LightningDataModule):
	def __init__(self, dataset, model_name, batch_size, max_prompt_length, num_workers):
		super().__init__()
		self.ds = dataset
		self.model_name = model_name
		self.batch_size = batch_size
		self.max_prompt_length = max_prompt_length
		self.num_workers = num_workers
		self.tokenizer = AutoTokenizer.from_pretrained(model_name)
		tokenizer.padding_side = "left"

	def setup(self, stage=None):
		self.train_dataset = self.ds["train"]
		self.val_dataset = self.ds["validation"]

	def _collate_fn(self, batch):
		prompts = [example["prompt"] for example in batch]
		encoding = self.tokenizer(
			prompts,
			return_tensors="pt",
			padding=True,
			truncation=True,
			max_length=self.max_prompt_length,
		)
		return {
			"prompt": prompts,
			"input_ids": encoding["input_ids"],
			"attention_mask": encoding["attention_mask"],
		}

	def train_dataloader(self):
		return DataLoader(
			self.train_dataset,
			batch_size=self.batch_size,
			shuffle=True,
			num_workers=self.num_workers,
			collate_fn=self._collate_fn,
		)

	def val_dataloader(self):
		return DataLoader(
			self.val_dataset,
			batch_size=self.batch_size,
			num_workers=self.num_workers,
			collate_fn=self._collate_fn,
		)

In [13]:
class ReinforceModel(LightningModule):
	def __init__(
		self, sft_model_name, reward_model_name, learning_rate, max_new_tokens, alpha, top_k, top_p,
	):
		super().__init__()
		self.save_hyperparameters()

		self.policy_model = AutoModelForCausalLM.from_pretrained(sft_model_name)
		self.reward_model = AutoModelForSequenceClassification.from_pretrained(reward_model_name)
		self.tokenizer = AutoTokenizer.from_pretrained(sft_model_name)
		self.tokenizer.padding_side = "left"

		for param in self.reward_model.parameters():
			param.requires_grad = False

		# Инициализация бейзлайна в виде скользящего среднего
		self.register_buffer("moving_avg_reward", torch.tensor(0.0))

	def configure_optimizers(self):
		optimizer = AdamW(self.policy_model.parameters(), lr=self.hparams.learning_rate)
		return optimizer

	@torch.no_grad()
	def generate_responses(self, input_ids, attention_mask):
		# генерация ответов policy модели
		outputs = self.policy_model.generate(
			input_ids=input_ids,
			attention_mask=attention_mask,
			max_new_tokens=self.hparams.max_new_tokens,
			do_sample=True,
			top_k=self.hparams.top_k,
			top_p=self.hparams.top_p,
			pad_token_id=self.tokenizer.pad_token_id,
		)

		responses = [out[len(input_ids[i]):] for i, out in enumerate(outputs)]
		return responses, outputs

	def compute_reward(self, full_input_ids, attention_mask):
		outputs = self.reward_model(input_ids=full_input_ids, attention_mask=attention_mask)
		# возьмем за награду вероятность положительного класса
		rewards = torch.sigmoid(outputs.logits).squeeze(-1)
		return rewards

	def training_step(self, batch, batch_idx):
		prompts = batch["prompt"]
		input_ids = batch["input_ids"]
		attention_mask = batch["attention_mask"]

		responses, full_outputs = self.generate_responses(input_ids, attention_mask)

		# Полные входы: prompt + response
		full_input_ids = full_outputs
		full_attention_mask = (full_input_ids != self.tokenizer.pad_token_id).long()

		rewards = self.compute_reward(full_input_ids, full_attention_mask).detach()
		self.moving_avg_reward = self.hparams.alpha * self.moving_avg_reward + (1 - self.hparams.alpha) * rewards.mean()

		outputs = self.policy_model(full_input_ids, attention_mask=full_attention_mask)
		logits = outputs.logits[:, :-1, :]
		targets = full_input_ids[:, 1:]

		log_probs = logits.log_softmax(dim=-1)
		selected_log_probs = torch.gather(log_probs, 2, targets.unsqueeze(-1)).squeeze(-1)

		gen_len = full_input_ids.size(1) - input_ids.size(1)
		mask = torch.zeros_like(selected_log_probs)
		mask[:, -gen_len:] = 1.0

		baseline = self.moving_avg_reward.detach()
		advantage = rewards.unsqueeze(1) - baseline
		loss = - (selected_log_probs * advantage * mask).sum(dim=1).mean()

		self.log("train_loss", loss, prog_bar=True)
		self.log("moving_avg_reward", self.moving_avg_reward, prog_bar=True)
		self.log("reward", rewards.mean(), prog_bar=True)

		return loss
	
	def validation_step(self, batch, batch_idx):
		pass


In [14]:
dm = ReinforceDataModule(
    ds,
    model_name="HuggingFaceTB/SmolLM2-135M-Instruct",
    batch_size=16,
    max_prompt_length=256,
    num_workers=8,
)

reinforce_model = ReinforceModel(
	sft_model_name = "HuggingFaceTB/SmolLM2-135M-Instruct",
	reward_model_name = "reward_model/checkpoint-439",
	learning_rate=5e-5,
	max_new_tokens=64,
	alpha=0.99,
	top_k=30,
	top_p=0.95,
)

logger = WandbLogger(
    name="smollm2-reinforce"
)

trainer = Trainer(
    max_epochs=3,
    accelerator="gpu",
    log_every_n_steps=10,
    logger=logger,
)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [15]:
trainer.fit(reinforce_model, datamodule=dm)

You are using a CUDA device ('NVIDIA GeForce RTX 4060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/hxrt_mx/ml/notebooks/smollm2-reinforce-alignment/.venv/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type                           | Params | Mode
-----------------------------------------------------------------------
0 | policy_model | LlamaForCausalLM               | 134 M  | eval
1 | reward_model | LlamaForSequenceClassification | 134 M  | eval
-

Epoch 0:   0%|          | 0/452 [00:00<?, ?it/s]                            

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Epoch 0:   0%|          | 1/452 [00:05<42:11,  0.18it/s, v_num=5dwa, train_loss=82.20, moving_avg_reward=0.00277, reward=0.277]

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

Epoch 2: 100%|█████████▉| 451/452 [1:14:31<00:09,  0.10it/s, v_num=5dwa, train_loss=0.00232, moving_avg_reward=0.976, reward=0.975]

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Epoch 2: 100%|██████████| 452/452 [1:14:35<00:00,  0.10it/s, v_num=5dwa, train_loss=-0.000108, moving_avg_reward=0.976, reward=0.978]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 452/452 [1:14:38<00:00,  0.10it/s, v_num=5dwa, train_loss=-0.000108, moving_avg_reward=0.976, reward=0.978]
